<a href="https://colab.research.google.com/github/olaviinha/NeuralStyleTransfer/blob/master/NeuralStyleTransfer_titu1994.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Neural Style Transfer <font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">Neural Style Transfer & Neural Doodles</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/NeuralStyleTransfer" target="_blank"><font color="#999" size="4">Github</font></a>

Colab and batch prosessor for [Neural Style Transfer & Neural Doodles](https://github.com/titu1994/Neural-Style-Transfer) by Somshubra Majumdar, an implementation of [A Neural Algorithm of Artistic Style](http://arxiv.org/abs/1508.06576). This notebook takes **directories** of content images and style images, and styles each content image with each style image, saving them to output directory. Enter all directory paths relative to your Google Drive root.

In [ ]:
#@title Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisities.

%tensorflow_version 1.x
force_setup = False

import os
from google.colab import output
import warnings
warnings.filterwarnings('ignore')

# Mount Drive
if not os.path.isdir('/content/drive') and force_setup == False:
  from google.colab import drive
  drive.mount('/content/drive')

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

!pip -q install import-ipynb
!curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

!pip -q install 'scipy<=1.2.1'

%cd /content/
!git clone https://github.com/titu1994/Neural-Style-Transfer.git
%cd /content/Neural-Style-Transfer

output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.')

In [ ]:
#@title Style transfer

content_images_dir = "" #@param {type:"string"}
styles_dir = "" #@param {type:"string"}
output_dir = "" #@param {type:"string"}
#network = "INetwork" #@param ["INetwork", "Network"]
image_size = 880 #@param {type:"slider", min:400, max:1200, step:20}

# Loss Weights
#def 0.025
content_weight = 0.01 #@param {type:"slider", min:0.01, max:1, step:0.01}
#def 1.0
style_weight = 1 #@param {type:"slider", min:0, max:1, step:0.01}
#def 1.0
style_scale = 1 #@param {type:"slider", min:0.1, max:50, step:0.1}
TOTAL_VARIATION_WEIGHT = 8.5e-5
CONTENT_LOSS_TYPE = 0

# Training arguments
iterations = 400 #@param {type:"slider", min:10, max:1000, step:10}
MODEL = 'vgg19'
rescale_image = False #@param {type:"boolean"}
maintain_aspect_ratio = True #@param {type:"boolean"}
  
# Transfer Arguments
CONTENT_LAYER = 'conv' + '5_2'  # only change the number 5_2 to something in a similar format
INITIALIZATION_IMAGE = 'content'
POOLING_TYPE = 'max'

drive_root = "/content/drive/MyDrive/"
content_images_dir = drive_root+content_images_dir
output_dir = drive_root+output_dir+"/"
styles_dir = drive_root+styles_dir+"/"
dir_path = '/content/Neural-Style-Transfer'
#NETWORK = network + '.py'
NETWORK = "INetwork.py"
IMAGE_SIZE = image_size
CONTENT_WEIGHT = content_weight
STYLE_WEIGHT = style_weight
STYLE_SCALE = style_scale
NUM_ITERATIONS = iterations
RESCALE_IMAGE = rescale_image
MIN_IMPROVEMENT = 0
preserve_color = False

if rescale_image is False: 
  RESCALE_IMAGE = 'false'
else: 
  RESCALE_IMAGE = 'true'
if maintain_aspect_ratio is False:
  MAINTAIN_ASPECT_RATIO = 'false'
else:
  MAINTAIN_ASPECT_RATIO = 'true'
if preserve_color is False:
  PRESERVE_COLOR = 'false'
else: 
  PRESERVE_COLOR = 'true'

import os, random, string, ntpath
from os import listdir
from os.path import isfile, join
from glob import glob

tmp = "/content/tmp"
if os.path.isdir(tmp):
  print('Clean up temporary files', end='.....')
  !rm -r {tmp}
  print('Done.')
print('Fetch content and style images from Drive', end='.....')
!mkdir {tmp}
!mkdir {tmp}/input
!mkdir {tmp}/styles
!mkdir {tmp}/output  
!cp -RT "{content_images_dir}" {tmp}/input
!cp -RT "{styles_dir}" {tmp}/styles
print('Done.')

content_images_dir = tmp+"/input"
styles_dir = tmp+"/styles"
output_dir_tmp = tmp+"/output"

def rndStr(length):
  letters = string.ascii_lowercase
  result_str = ''.join(random.choice(letters) for i in range(length))
  return result_str

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

#@title Style transfer

%matplotlib inline
import matplotlib.pyplot as plt

content_imgs = [f for f in listdir(content_images_dir) if isfile(join(content_images_dir, f))]
style_imgs = [f for f in listdir(styles_dir) if isfile(join(styles_dir, f))]

content_imgs.sort()
style_imgs.sort()

for content_img in content_imgs:

  content_img_leaf = content_img
  content_img = content_images_dir+"/"+content_img
  # CONTENT IMG

  CONTENT_IMAGE_FN = content_img #list(content_img)[0]
  CONTENT_IMAGE_FN_temp = CONTENT_IMAGE_FN.strip().replace(" ", "_")

  if CONTENT_IMAGE_FN != CONTENT_IMAGE_FN_temp:
    os.rename(CONTENT_IMAGE_FN, CONTENT_IMAGE_FN_temp)
    CONTENT_IMAGE_FN = CONTENT_IMAGE_FN_temp
    
  #print("Content image filename :", CONTENT_IMAGE_FN)

  # %matplotlib inline
  # import matplotlib.pyplot as plt

  # fig = plt.figure(figsize=(10, 10))
  # img = plt.imread(CONTENT_IMAGE_FN)
  # plt.axis('off')
  # plt.title('Content image')
  # plt.imshow(img)

  # STYLE IMG
  for style_img in style_imgs:
    
    print('>>> Styling content', content_img_leaf, 'with style', style_img)
    style_img = styles_dir+"/"+style_img
    STYLE_IMAGE_FN = style_img #list(style_img)[0]
    STYLE_IMAGE_FN_temp = STYLE_IMAGE_FN.strip().replace(" ", "_")

    if STYLE_IMAGE_FN != STYLE_IMAGE_FN_temp:
      os.rename(STYLE_IMAGE_FN, STYLE_IMAGE_FN_temp)
      STYLE_IMAGE_FN = STYLE_IMAGE_FN_temp
      
    #print("Style image filename :", STYLE_IMAGE_FN)

    fig = plt.figure(figsize=(10, 10))
    img = plt.imread(STYLE_IMAGE_FN)
    plt.axis('off')
    plt.title('Style image')
    plt.imshow(img)

    RESULT_DIR = output_dir_tmp
    RESULT_PREFIX = RESULT_DIR + "/NST_" + slug(basename(CONTENT_IMAGE_FN_temp)) + '_' + slug(basename(style_img)) + '__' + rndStr(4)
    FINAL_IMAGE_PATH = RESULT_PREFIX + "_%d.png" % (NUM_ITERATIONS)

    if not os.path.exists(RESULT_DIR):
      os.makedirs(RESULT_DIR)

    !python {dir_path}/{NETWORK} {CONTENT_IMAGE_FN} {STYLE_IMAGE_FN} {RESULT_PREFIX} \
      --image_size {IMAGE_SIZE} --content_weight {CONTENT_WEIGHT} --style_weight \
      {STYLE_WEIGHT} --style_scale {STYLE_SCALE} --total_variation_weight \
      {TOTAL_VARIATION_WEIGHT} --content_loss_type {CONTENT_LOSS_TYPE} --num_iter \
      {NUM_ITERATIONS} --model {MODEL} --rescale_image {RESCALE_IMAGE} \
      --maintain_aspect_ratio {MAINTAIN_ASPECT_RATIO} --content_layer {CONTENT_LAYER} \
      --init_image {INITIALIZATION_IMAGE} --pool_type {POOLING_TYPE} --preserve_color \
      {PRESERVE_COLOR} --min_improvement {MIN_IMPROVEMENT}

    #output_imgs = [f for f in listdir(output_dir_tmp) if isfile(join(output_dir_tmp, f))]
    output_imgs = glob(output_dir_tmp+"/*.png")
    for output_img in output_imgs:
      cmpx = 'iteration_'+str(iterations)
      if cmpx in output_img:
        file = path_leaf(output_img)
        print(">>> Save", file, 'to Drive')
        !cp -RT {output_img} "{output_dir}/{file}"
        
        fig = plt.figure(figsize=(10, 10))
        img = plt.imread(output_img)
        plt.axis('off')
        plt.title('Generated image')
        plt.imshow(img)

